In [210]:
from given import *
import re

In [66]:
pd.set_option('display.max_colwidth', 115)

In [115]:
_N_ = pd.read_pickle(big_data/'trans/N10.pkl') #get_N()

In [116]:
N = _N_[('2009-01-01'<=_N_.time)&(_N_.time<'2009-02-01')]

# Helpers

In [89]:
ljust = lambda s: s.str.ljust(s.str.len().max())

In [97]:
def sample(k):
    def sample_k(df):
        n = len(a)
        samps = sorted(random.sample(range(n), k))
        return df.iloc[samps]
    return sample_k

# Notes and TODOs

In [ ]:
{
    "Royal Bank of Canada": "RBC"
    "Telefonaktiebolaget LM Ericsson": "Ericsson",
    "Mobil\'nye Telesistemy PAO": "MTS",
    "INTL FCStone Inc": ["International Assets Holding Corp", "FCStone Group"]
    "NFP Corp": "National Financial Partners",
    "NXXI Inc": "Nutrition 21 Inc",
    "CNX Gas Corp": "Consol Energy Inc",
    "WEX Inc": "Wright Express Corp",
    "LRAD Corp": "American Technology Corp",
    "O I Corp": "OI Corp"
};

In [ ]:
'STCKa.O vs STCKb.O';

In [ ]:
'''
stock matching algorithm:
    match whole
    loop take away one token from right and try match
    repeat above with symbols removed (add no space)
'''

In [ ]:
'''
preprocessing pipeline:
    convert numbers
    convert stock ticker symbols
    replace ` => '
    replace \ => [delete]
    replace ... => #ellipsis
    replace . => [space]
    replace -- => #dash
    lower
    match stock name
    
''';

# Preprocessing

## real code

#### define some constants

In [ ]:
endings = 'Inc AG Corp PLC LP Co Ltd SA NV ASA LLC A/S PAO'

#### simple subs

In [231]:
class resub(O()):
    class Money(O()):
        re = re.compile(r'\$[0-9]+.?[0-9]*')
    class Year(O()):
        re = re.compile('('+'|'.join(str(i) for i in range(1999, 2022))+')')
    class Decimal(O()):
        re = re.compile(r'[0-9]+.[0-9]*')
    class Count(O()):
        re = re.compile(r'[0-9]+')
        fn = lambda match: '#count' + str(len(match.string) if match.string[0] not in '01' else len(match.string)-1)
    class Ticker(O()):
        

#### match company names

## just looking

In [183]:
for name in _N_.assetName.unique():
    print(name)

PetroChina Co Ltd
Travelers Companies Inc
Wal-Mart Stores Inc
Google Inc
XM Satellite Radio Holdings Inc
Sirius XM Radio Inc
Walt Disney Co
Microsoft Corp
Yahoo! Inc
Tenet Healthcare Corp
Siebert Financial Corp
Monsanto Co
Constellation Brands Inc
Rite Aid Corp
Cyren Ltd
Qualcomm Inc
Siemens AG
Internet Gold Golden Lines Ltd
Goldman Sachs Group Inc
Merrill Lynch & Co Inc
Citigroup Inc
General Electric Co
Time Warner Inc
Twenty-First Century Fox Inc
Viacom Inc
Sony Corp
BHP Billiton Ltd
Ryanair Holdings PLC
Fomento Economico Mexicano SAB de CV
Sanofi SA
AU Optronics Corp
Taiwan Semiconductor Manufacturing Co Ltd
Royal Bank of Scotland Group PLC
Hutchison Telecommunications International Ltd
Indosat Tbk PT
Korea Electric Power Corp
Nasdaq Inc
Kookmin Bank
Woori Finance Holdings Co Ltd
Honda Motor Co Ltd
LG Display Co Ltd
Apple Inc
Adobe Systems Inc
China Mobile Ltd
HSBC Holdings PLC
Willis Towers Watson PLC
Banco Santander SA
Daimler AG
Titanium Metals Corp
Companhia Siderurgica Nacional

Tellurian Inc
Del Monte Foods Co
Borders Group Inc
Allergan Finance LLC
Medifast Inc
KP Pharmaceuticals LLC
Jack in the Box Inc
Cigna Corp
Kelly Services Inc
SFN Group LLC
Hope Bancorp Inc
McEwen Mining Inc
SQBG Inc
FEI Co
Endo Health Solutions Inc
Hungarian Telephone And Cable Corp
Newmont Mining Corp
Here Holding Corp
Mair Holdings Inc
NiSource Inc
Astora Women's Health Holdings LLC
Mountain Province Diamonds Inc
Oil States International Inc
Lifetime Brands Inc
Schmitt Industries Inc
Fleetwood Enterprises Inc
A. O. Smith Corp
Western Union Co
Imperial Oil Ltd
Pepco Holdings LLC
Ingredion Inc
Unisys Corp
SAP AG
BEA Systems Inc
Sonde Resources Corp
Occidental Petroleum Corp
Buca Inc
Abaxis Inc
VF Corp
Grupo Aeroportuario del Pacifico SAB de CV
Colonial Properties Trust
Mentor Graphics Corp
LSI Industries Inc
Rexnord-Zurn Holdings Inc
Microsemi Corp Memory and Storage Solutions
Taseko Mines Ltd
Associated Banc-Corp
Potlatch Corp
Compania Cervecerias Unidas SA
Polymet Mining Corp
Central

Butler National Corp
Hemagen Diagnostics Inc
Gungnir Resources Inc
Avigen Inc
Electro Rent Corp
MGT Capital Investments Inc
Ultrapetrol (Bahamas) Ltd
NII Holdings Inc
C&F Financial Corp
Edwards Lifesciences Corp
Donaldson Company Inc
Veramark Technologies Inc
SRC Liquidation LLC
DNP Select Income Fund Inc
American Medical Alert Corp
Rand Capital Corp
Minrad International Inc
Glen Rose Petroleum Corp
Delcath Systems Inc
Cleco Corporate Holdings LLC
Genlyte Group Inc
Alliance HealthCare Services Inc
Yadkin Financial Corp
LL&E Royalty Trust
Tortoise Energy Infrastructure Corp
Tortoise North American Energy Corp
Camco Financial Corp
Quality Distribution Inc
Britton & Koontz Capital Corp
Zilog Inc
Champion Enterprises Inc
Sturm Ruger & Company Inc
TIB Financial Corp
Datawatch Corp
Integrated Silicon Solution Inc
MTM Technologies Inc
Magyar Bancorp Inc
Farmers National Bancorp
Smart & Final LLC
North Central Bancshares Inc
Catalina Marketing Corp
Eagle Rock Energy Partners LP
Devcon Internat

Northwest Airlines Corp
Clean Energy Fuels Corp
LDK Solar Co Ltd
Fuwei Films Holdings Co Ltd
PimCo Municipal Advantage Fund InCorp
PIMCO Income Strategy Fund II
PIMCO Income Strategy Fund
Allianzgi Convertible & Income Fund
Allianzgi Convertible & Income Fund II
PIMCO New York Municipal Income Fund II
PIMCO California Municipal Income Fund II
PIMCO Municipal Income Fund III
PIMCO California Municipal Income Fund Iii
Pimco New York Municipal Income Fund III
PIMCO Municipal Income Fund
PIMCO California Municipal Income Fund
PIMCO New York Municipal Income Fund
Nuveen Core Equity Alpha Fund
Nuveen Diversified Currency Opportunities Fund
Alpine Global Dynamic Dividend Fund
Gpc Biotech AG
TomoTherapy Inc
Neuro-Hitech Inc
Response Genetics Inc
Starent Networks LLC
WaferGen Bio-Systems Inc
Towerstream Corp
DGT Holdings Corp
Florida Gaming Corp
Infinera Corp
Deutsche Global High Income Fund Inc
FBR & Co
Yingli Green Energy Holding Co Ltd
Limelight Networks Inc
Boss Holdings Inc
Chang-On Intern

Palo Alto Networks Inc
Chuy's Holdings Inc
Natural Grocers By Vitamin Cottage Inc
Northern Tier Energy LP
E2open LLC
Horizon Therapeutics Inc
Del Frisco's Restaurant Group Inc
Globus Medical Inc
Howard Bancorp Inc
Eloqua Inc
Xylem Inc
Bloomin' Brands Inc
Peregrine Semiconductor Corp
Tronox Ltd
Manchester United PLC
Performant Financial Corp
Engility Holdings Inc
Hi Crush Partners LP
CU Bancorp
Sandstorm Gold Ltd
IF Bancorp Inc
Seven Arts Entertainment Inc
Trulia LLC
National Bank Holdings Corp
Capital Bank Financial Corp
Emerald Oil Inc
Corts Trust Weyerhauser Co
Pareteum Corp
Qualys Inc
Summit Midstream Partners LP
ADT Corp
Kraft Foods Group Inc
LifeLock Inc
JAVELIN Mortgage Investment Corp
Tile Shop Holdings Inc
W. P. Carey Inc
Berry Global Group Inc
Regulus Therapeutics Inc
Fleetmatics Group Ltd
Blackrock Municipal 2030 Target Term Trust
Ryman Hospitality Properties Inc
Amira Nature Foods Ltd
Realogy Holdings Corp
RLJ Entertainment Inc
Shutterstock Inc
Kythera Biopharmaceuticals Inc

In [208]:
_N_.pipe(lambda N: N.headline[N.assetName=='Apple Inc'].pipe(ljust))

62         PRESS DIGEST - Washington Post Business - Dec 29                                                               ...
110        PRESS DIGEST - Wall Street Journal - Jan 2                                                                     ...
703        UPDATE 1-U.S. securities class-actions down sharply-study                                                      ...
951        PRESS DIGEST - New York Times business - Jan 3                                                                 ...
6367       HEADLINE STOCKS - U.S. stocks to watch Jan. 5                                                                  ...
6382       HEADLINE STOCKS - U.S. stocks to watch Jan. 5                                                                  ...
6910       HEADLINE STOCKS - U.S. stocks on the move on Jan. 5                                                            ...
6920       HEADLINE STOCKS - U.S. stocks on the move on Jan. 5                                                        

# Explore

In [94]:
/pd.DataFrame sorted(set(zip(M.assetCode,M.assetName)))

,0,1
0,A.N,Agilent Technologies Inc
1,AA.N,Alcoa Corp
2,AAI.N,AirTran Holdings Inc
3,AAL.O,American Airlines Group Inc
4,AAMRQ.OB,Unknown
5,AAN.N,Aaron's Inc
6,AAON.O,Aaon Inc
7,AAP.N,Advance Auto Parts Inc
8,AAPL.O,Apple Inc
9,AAT.N,American Assets Trust Inc


In [119]:
N[N.assetNamesCount==2].headline.pipe(ljust)

1521196    SNAPSHOT - Financial Crisis - 0045 GMT                                                                         ...
1521197    SNAPSHOT - Financial Crisis - 0045 GMT                                                                         ...
1521199    UPDATE 2-GM gets $4 bln rescue loan, Chrysler on hold                                                          ...
1521200    UPDATE 2-GM gets $4 bln rescue loan, Chrysler on hold                                                          ...
1521202    RPT-India's TCS sees back office deals                                                                         ...
1521203    RPT-India's TCS sees back office deals                                                                         ...
1521204    IBM in sales alliance with Japan's Ricoh - paper                                                               ...
1521205    IBM in sales alliance with Japan's Ricoh - paper                                                           

In [150]:
N.loc[[1527026,1527027]]

,time,sourceTimestamp,firstCreated,sourceId,headline,urgency,takeSequence,provider,subjects,audiences,bodySize,companyCount,headlineTag,marketCommentary,sentenceCount,wordCount,assetCodes,assetName,firstMentionSentence,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive,sentimentWordCount,noveltyCount12H,noveltyCount24H,noveltyCount3D,noveltyCount5D,noveltyCount7D,volumeCounts12H,volumeCounts24H,volumeCounts3D,volumeCounts5D,volumeCounts7D,assetNameId,assetNamesCount,firstMentionFixed,groupbyTime
1527026,2009-01-07 04:10:50+00:00,2009-01-07 04:10:50+00:00,2009-01-07 04:10:50+00:00,06a6b51052ef9c67,UPDATE 2-Bank of America sells $2.8 bln CCB stake,3,1,RTRS,"{'MRG', 'DE', 'ISU', 'FINS', 'INVS', 'SG', 'COFS', 'RTRS', 'MEVN', 'EMRG', 'BUS', 'BNK', 'FIN', 'FUND', 'EUROPE...","{'Z', 'UKI', 'T', 'DNP', 'PSC', 'U', 'D', 'M', 'EMK', 'RNP', 'NAW', 'PTD', 'E'}",3737,4,UPDATE 2,False,27,709,"{'UBS.N', 'UBSN.DE', 'UBSN.VX', 'UBSN.F'}",UBS AG,15,0.09759,1,0.112890,0.159270,0.727840,97,0,0,0,0,0,13,23,32,36,56,8948,2,15,2009-01-07 22:00:00+00:00
1527027,2009-01-07 04:10:50+00:00,2009-01-07 04:10:50+00:00,2009-01-07 04:10:50+00:00,06a6b51052ef9c67,UPDATE 2-Bank of America sells $2.8 bln CCB stake,3,1,RTRS,"{'MRG', 'DE', 'ISU', 'FINS', 'INVS', 'SG', 'COFS', 'RTRS', 'MEVN', 'EMRG', 'BUS', 'BNK', 'FIN', 'FUND', 'EUROPE...","{'Z', 'UKI', 'T', 'DNP', 'PSC', 'U', 'D', 'M', 'EMK', 'RNP', 'NAW', 'PTD', 'E'}",3737,4,UPDATE 2,False,27,709,"{'IKM.N', 'IKL.N', 'BAC.N', 'IKJ.N', 'IKR.N'}",Bank of America Corp,1,1.00000,-1,0.722047,0.184978,0.092975,581,4,4,4,4,4,18,19,50,62,100,141,2,1,2009-01-07 22:00:00+00:00
